In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd drive/MyDrive/Github/ml-blog

/content/drive/MyDrive/Github/ml-blog


In [ ]:
!pip install xlrd==1.2.0
!pip install matplotlib==3.5.3

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xlrd

In [4]:
filename = 'default of credit card clients.xls'

In [5]:
DATA = os.path.relpath('/content/drive/MyDrive/Github/ml-blog/credit/data/' + filename)

In [6]:
df = pd.read_excel(DATA, 'Data', index_col=[0], header=[1], na_values='NA')
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [8]:
X = df.iloc[:, :22]
y = df.iloc[:, 23]

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
lr_pipe = Pipeline([
    ('pca', PCA(n_components=2)),
     ('logreg', LogisticRegression(random_state=42))]
)

In [34]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA(n_components=2)),
                ('logreg', LogisticRegression(random_state=42))])

In [35]:
lr_pipe.score(X_test, y_test)

0.5178333333333334

In [36]:
y_pred_lr = lr_pipe.predict(X_test)

In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.46      0.60      4687
           1       0.27      0.72      0.39      1313

    accuracy                           0.52      6000
   macro avg       0.56      0.59      0.50      6000
weighted avg       0.73      0.52      0.55      6000

